In [43]:
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", None)

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, classification_report

from itertools import combinations

from IPython.display import display

## Load Dataset

In [16]:
url = "https://breathecode.herokuapp.com/asset/internal-link?id=2325&path=adult-census-income.csv"
df = pd.read_csv(url)

df.head(50)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
6,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K
7,74,State-gov,88638,Doctorate,16,Never-married,Prof-specialty,Other-relative,White,Female,0,3683,20,United-States,>50K
8,68,Federal-gov,422013,HS-grad,9,Divorced,Prof-specialty,Not-in-family,White,Female,0,3683,40,United-States,<=50K
9,41,Private,70037,Some-college,10,Never-married,Craft-repair,Unmarried,White,Male,0,3004,60,?,>50K


In [17]:
df.shape

(32561, 15)

In [ ]:
df.info()

In [18]:
df.describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [19]:
df["income"].value_counts(normalize=True)

income
<=50K    0.75919
>50K     0.24081
Name: proportion, dtype: float64

In [20]:
df["workclass"].value_counts(normalize=True)

workclass
Private             0.697030
Self-emp-not-inc    0.078038
Local-gov           0.064279
?                   0.056386
State-gov           0.039864
Self-emp-inc        0.034274
Federal-gov         0.029483
Without-pay         0.000430
Never-worked        0.000215
Name: proportion, dtype: float64

In [21]:
df["occupation"].value_counts(normalize=True)


occupation
Prof-specialty       0.127146
Craft-repair         0.125887
Exec-managerial      0.124873
Adm-clerical         0.115783
Sales                0.112097
Other-service        0.101195
Machine-op-inspct    0.061485
?                    0.056601
Transport-moving     0.049046
Handlers-cleaners    0.042075
Farming-fishing      0.030527
Tech-support         0.028500
Protective-serv      0.019932
Priv-house-serv      0.004576
Armed-Forces         0.000276
Name: proportion, dtype: float64

In [22]:
df["native.country"].value_counts(normalize=True)

native.country
United-States                 0.895857
Mexico                        0.019748
?                             0.017905
Philippines                   0.006081
Germany                       0.004207
Canada                        0.003716
Puerto-Rico                   0.003501
El-Salvador                   0.003255
India                         0.003071
Cuba                          0.002918
England                       0.002764
Jamaica                       0.002488
South                         0.002457
China                         0.002303
Italy                         0.002242
Dominican-Republic            0.002150
Vietnam                       0.002058
Guatemala                     0.001966
Japan                         0.001904
Poland                        0.001843
Columbia                      0.001812
Taiwan                        0.001566
Haiti                         0.001351
Iran                          0.001321
Portugal                      0.001136
Nicaragua 

In [23]:
pd.crosstab(df["native.country"], df["income"], normalize="index")

income,<=50K,>50K
native.country,,
?,0.749571,0.250429
Cambodia,0.631579,0.368421
Canada,0.677686,0.322314
China,0.733333,0.266667
Columbia,0.966102,0.033898
Cuba,0.736842,0.263158
Dominican-Republic,0.971429,0.028571
Ecuador,0.857143,0.142857
El-Salvador,0.915094,0.084906


In [24]:
# find columns that have ?
(df == "?").sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

## Step 2: Data preprocessing. Clean null or misencoded data, transform categorical variables, and normalize numerical variables.

In [25]:
# looking at only the columns with ?
question_counts = (df == "?").sum()
question_counts[question_counts > 0]

workclass         1836
occupation        1843
native.country     583
dtype: int64

In [26]:
# percentage of ? in those columns
question_percent = (df == "?").mean() * 100
question_percent[question_percent > 0]

workclass         5.638647
occupation        5.660146
native.country    1.790486
dtype: float64

In [27]:
# change ? in all columns to na
df = df.replace("?", pd.NA)

In [28]:
df.isna().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

In [29]:
df["native.country"].isna().sum()

np.int64(583)

In [30]:
pd.crosstab(df["native.country"], df["income"], normalize="index", dropna=False)

income,<=50K,>50K
native.country,,
Cambodia,0.631579,0.368421
Canada,0.677686,0.322314
China,0.733333,0.266667
Columbia,0.966102,0.033898
Cuba,0.736842,0.263158
Dominican-Republic,0.971429,0.028571
Ecuador,0.857143,0.142857
El-Salvador,0.915094,0.084906
England,0.666667,0.333333


In [31]:
# drop sample weight fnlwgt
df = df.drop(columns=["fnlwgt"])

In [32]:
# precheck on na in native.country
df["native.country"].isna().sum()

np.int64(583)

In [33]:
# drop na native.country
df = df.dropna(subset=["native.country"]).copy()

In [34]:
# confirm dropped
df["native.country"].isna().sum()

np.int64(0)

In [35]:
# set X and y (target)
X = df.drop(columns=["income"])
y = df["income"]

In [36]:
# turn y to binary
y = y.apply(lambda x: 1 if x == ">50K" else 0)

In [37]:
y.value_counts(normalize=True)

income
0    0.759366
1    0.240634
Name: proportion, dtype: float64

In [38]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object", "string"]).columns

In [39]:
num_cols

Index(['age', 'education.num', 'capital.gain', 'capital.loss',
       'hours.per.week'],
      dtype='str')

In [40]:
cat_cols

Index(['workclass', 'education', 'marital.status', 'occupation',
       'relationship', 'race', 'sex', 'native.country'],
      dtype='str')

In [41]:
# creates a preprocessing transformer that will scale numerical features and one-hot encode categorical features when applied
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

In [42]:
preprocessor

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

## Create Baseline Model

In [ ]:
# create splits
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
# create pipeline to run the preprocessing transformer through
model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", LogisticRegression(max_iter=2000))
])

In [46]:
model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contai

In [47]:
model.score(X_test, y_test)

0.8542839274546592

In [ ]:
# apply predict to model
y_pred = model.predict(X_test)

In [50]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8542839274546592
              precision    recall  f1-score   support

           0       0.88      0.93      0.91      4857
           1       0.74      0.60      0.66      1539

    accuracy                           0.85      6396
   macro avg       0.81      0.77      0.79      6396
weighted avg       0.85      0.85      0.85      6396



## 3. Define the recommendation problem. Plan how you will structure your recommendation system:

- What is being recommended?

    - The system recommends actionable changes (such as increased education level, changes in occupation, or increased working hours) that increase the predicted probability of earning more than $50,000 per year.

- Who is the "user" in this case?

    - The user is an individual whose demographic and socioeconomic attributes are provided as input to the model. The system focuses primarily on individuals predicted to have a lower probability of earning more than $50,000.

- What variables define a user's profile?

    - A user’s profile is defined by all model features retained after preprocessing, including:
      age, workclass, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, and native-country.

## 4. CONTENT-BASED FILTERING SECTION

In [51]:
# transform full dataset
X_vector = preprocessor.fit_transform(X)

In [52]:
# seperate high earners
high_earners_idx = np.where(y == 1)[0]

X_high = X_vector[high_earners_idx]

In [53]:
# take first person as example and create a sample user
user_profile = X.iloc[0:1]   

In [54]:
# transform sample user
user_vector = preprocessor.transform(user_profile)

In [55]:
# compute cosine similarity
similarities = cosine_similarity(user_vector, X_high)

In [56]:
# find the 5 most similar high earners
top_k = 5
top_indices = similarities[0].argsort()[-top_k:][::-1]

similar_high_earners = X.iloc[high_earners_idx[top_indices]]

In [57]:
print("USER PROFILE:")
print(user_profile)

USER PROFILE:
   age workclass education  education.num marital.status occupation  \
0   90       NaN   HS-grad              9        Widowed        NaN   

    relationship   race     sex  capital.gain  capital.loss  hours.per.week  \
0  Not-in-family  White  Female             0          4356              40   

  native.country  
0  United-States  


In [58]:
print("\nSIMILAR HIGH EARNERS:")
print(similar_high_earners)


SIMILAR HIGH EARNERS:
     age         workclass     education  education.num      marital.status  \
130   71           Private       HS-grad              9  Married-civ-spouse   
36    53           Private       HS-grad              9       Never-married   
180   57           Private  Some-college             10             Widowed   
34    60  Self-emp-not-inc       HS-grad              9       Never-married   
691   61               NaN       HS-grad              9  Married-civ-spouse   

          occupation   relationship   race     sex  capital.gain  \
130  Exec-managerial        Husband  White    Male             0   
36             Sales  Not-in-family  White    Male             0   
180     Adm-clerical      Unmarried  White  Female             0   
34   Exec-managerial  Not-in-family  Black    Male             0   
691              NaN        Husband  White    Male             0   

     capital.loss  hours.per.week native.country  
130          2377              45  United-

In [59]:
recommendations = {}

for col in ["education", "occupation", "workclass", "hours.per.week"]:
    
    user_value = user_profile[col].values[0]
    common_value = similar_high_earners[col].mode()[0]
    
    if user_value != common_value:
        recommendations[col] = common_value

recommendations

{'occupation': 'Exec-managerial',
 'workclass': 'Private',
 'hours.per.week': np.int64(38)}

In [97]:
print("Income Profile Analysis:\n")

current_prob = model.predict_proba(user_profile)[0][1]
print(f"Current probability of earning >50K: {current_prob:.2f}\n")

for col in ["education", "occupation", "workclass", "hours.per.week"]:
    
    user_value = user_profile[col].values[0]
    common_value = similar_high_earners[col].mode()[0]
    
    # GOOD MATCH
    if user_value == common_value:
        print(f"Your '{col}' is aligned with similar high earners ({user_value}).")
    
    # NEEDS IMPROVEMENT
    else:
        if col == "occupation":
            print(f"- Consider transitioning into roles like '{common_value}', which are common among similar high earners.")
        
        elif col == "workclass":
            print(f"- High earners are often in the '{common_value}' sector.")
        
        elif col == "hours.per.week":
            print(f"- Increasing working hours toward approximately {int(common_value)} hours per week may improve income probability.")
        
        elif col == "education":
            print(f"- Pursuing education at the '{common_value}' level may increase earning potential.")

Income Profile Analysis:

Current probability of earning >50K: 0.44

Your 'education' is aligned with similar high earners (HS-grad).
- Consider transitioning into roles like 'Exec-managerial', which are common among similar high earners.
- High earners are often in the 'Private' sector.
- Increasing working hours toward approximately 38 hours per week may improve income probability.


In [61]:
current_prob = model.predict_proba(user_profile)[0][1]

In [63]:
recommendation_effects = []

for col in ["education", "occupation", "workclass", "hours.per.week"]:
    
    user_value = user_profile[col].values[0]
    common_value = similar_high_earners[col].mode()[0]
    
    # Only test if different
    if user_value != common_value:
        
        modified_user = user_profile.copy()
        modified_user[col] = common_value
        
        new_prob = model.predict_proba(modified_user)[0][1]
        
        improvement = new_prob - current_prob
        
        recommendation_effects.append({
            "feature": col,
            "recommended_value": common_value,
            "new_probability": new_prob,
            "improvement": improvement
        })

In [ ]:
# counterfactual recommendation analysis
recommendation_df = pd.DataFrame(recommendation_effects)

recommendation_df = recommendation_df.sort_values(
    by="improvement",
    ascending=False
)

recommendation_df

,feature,recommended_value,new_probability,improvement
0,occupation,Exec-managerial,0.716533,0.273092
1,workclass,Private,0.522250,0.078810
2,hours.per.week,38,0.428591,-0.014849


In [ ]:
print("Ranked Income Improvement Recommendations:\n")

print(f"Current Probability of >50K: {current_prob:.2f}\n")

for _, row in recommendation_df.iterrows():
    print(f"- Changing '{row['feature']}' to '{row['recommended_value']}' "
          f"increases probability by {row['improvement']:.2f} "
          f"(new probability: {row['new_probability']:.2f})")

📊 Ranked Income Improvement Recommendations:

Current Probability of >50K: 0.44

- Changing 'occupation' to 'Exec-managerial' increases probability by 0.27 (new probability: 0.72)
- Changing 'workclass' to 'Private' increases probability by 0.08 (new probability: 0.52)
- Changing 'hours.per.week' to '38' increases probability by -0.01 (new probability: 0.43)


## Now lets make a function to run this on any user

In [ ]:
# function combining all the above report types to 1 nice report for any user
def full_income_analysis(
    user_input,
    model,
    preprocessor,
    X,
    y,
    top_k_neighbors=5,
    actionable_cols=("education", "occupation", "workclass", "hours.per.week"),
    max_combo_size=2
):

    # ---------------------------------
    # Normalize Input
    # ---------------------------------
    if isinstance(user_input, int):
        user_profile = X.iloc[[user_input]].copy()
    elif isinstance(user_input, dict):
        user_profile = pd.DataFrame([user_input])
    elif isinstance(user_input, pd.DataFrame):
        user_profile = user_input.copy()
    else:
        raise ValueError("User input must be index, dict, or 1-row DataFrame.")

    print("\n==============================")
    print("📊 FULL INCOME ADVISORY REPORT")
    print("==============================\n")

    # ---------------------------------
    # Current Prediction
    # ---------------------------------
    current_prob = model.predict_proba(user_profile)[0][1]
    current_pred = model.predict(user_profile)[0]

    print(f"Predicted Probability of >50K: {current_prob:.2f}\n")

    # ---------------------------------
    # Already High Income Branch
    # ---------------------------------
    if current_pred == 1:
        print("🎉 Your profile aligns with higher-income earners.")
        print("No immediate structural changes required.\n")
        print("==============================\n")
        return None

    # ---------------------------------
    # Find Similar High Earners
    # ---------------------------------
    X_vec = preprocessor.transform(X)
    high_idx = np.where(np.asarray(y) == 1)[0]
    X_high = X_vec[high_idx]

    user_vec = preprocessor.transform(user_profile)
    sims = cosine_similarity(user_vec, X_high)[0]
    top_neighbor_pos = sims.argsort()[-top_k_neighbors:][::-1]
    similar_high = X.iloc[high_idx[top_neighbor_pos]]

    # ---------------------------------
    # 1️⃣ Profile Alignment
    # ---------------------------------
    print("🔍 Profile Alignment:\n")

    for col in actionable_cols:
        user_val = user_profile[col].values[0]
        common_val = similar_high[col].mode(dropna=True)

        if len(common_val) == 0:
            continue

        common_val = common_val.iloc[0]

        if user_val == common_val:
            print(f"✔ {col}: aligned with high earners ({user_val})")
        else:
            print(f"✖ {col}: differs from high earners (common: {common_val})")

    # ---------------------------------
    # 2️⃣ Single-Feature Counterfactual
    # ---------------------------------
    print("\n📈 Single-Feature Impact Analysis:\n")

    single_results = []

    for col in actionable_cols:
        user_val = user_profile[col].values[0]
        common_val = similar_high[col].mode(dropna=True)

        if len(common_val) == 0:
            continue

        common_val = common_val.iloc[0]

        if user_val != common_val:
            modified = user_profile.copy()
            modified[col] = common_val

            new_prob = model.predict_proba(modified)[0][1]
            improvement = new_prob - current_prob

            single_results.append({
                "feature": col,
                "recommended_value": common_val,
                "new_probability": new_prob,
                "improvement": improvement
            })

    single_df = pd.DataFrame(single_results)

    if not single_df.empty:
        single_df = single_df.sort_values("improvement", ascending=False)
        for _, row in single_df.iterrows():
            print(
                f"- Change {row['feature']} → {row['recommended_value']} "
                f"(+{row['improvement']:.2f}, new prob: {row['new_probability']:.2f})"
            )
    else:
        print("No meaningful single-feature improvements identified.")

    # ---------------------------------
    # 3️⃣ Multi-Feature Simulation
    # ---------------------------------
    print("\n🚀 Multi-Feature Optimization:\n")

    multi_results = []

    candidate_map = {
        col: similar_high[col].mode(dropna=True).iloc[0]
        for col in actionable_cols
        if len(similar_high[col].mode(dropna=True)) > 0
        and user_profile[col].values[0] != similar_high[col].mode(dropna=True).iloc[0]
    }

    cols_available = list(candidate_map.keys())

    for r in range(2, max_combo_size + 1):
        for combo in combinations(cols_available, r):
            modified = user_profile.copy()

            for col in combo:
                modified[col] = candidate_map[col]

            new_prob = model.predict_proba(modified)[0][1]
            improvement = new_prob - current_prob

            multi_results.append({
                "change_set": combo,
                "new_probability": new_prob,
                "improvement": improvement
            })

    multi_df = pd.DataFrame(multi_results)

    if not multi_df.empty:
        multi_df = multi_df.sort_values("improvement", ascending=False)
        best = multi_df.iloc[0]

        print(
            f"Best Combined Strategy: {best['change_set']} "
            f"(+{best['improvement']:.2f}, new prob: {best['new_probability']:.2f})"
        )
    else:
        print("No strong multi-feature improvement identified.")

    # ---------------------------------
    # Structured Table Output
    # ---------------------------------
    print("\n📊 Structured Impact Summary:\n")

    if not single_df.empty:
        display(single_df.reset_index(drop=True))
    else:
        print("No single-feature improvements available.")

    if not multi_df.empty:
        print("\n📊 Multi-Feature Combination Summary:\n")
        display(multi_df.reset_index(drop=True))
    else:
        print("\nNo multi-feature improvements available.")

    print("\n==============================\n")

    return None

In [ ]:
# function to call the report with index id
def run_income_report_by_index(user_index):
    print(f"\nRunning Income Advisory Report for Dataset User Index: {user_index}\n")
    full_income_analysis(user_index, model, preprocessor, X, y)

Now lets run some users

In [90]:
run_income_report_by_index(26)


Running Income Advisory Report for Dataset User Index: 26


📊 FULL INCOME ADVISORY REPORT

Predicted Probability of >50K: 0.09

🔍 Profile Alignment:

✖ education: differs from high earners (common: 11th)
✔ occupation: aligned with high earners (Transport-moving)
✖ workclass: differs from high earners (common: Private)
✖ hours.per.week: differs from high earners (common: 40)

📈 Single-Feature Impact Analysis:

- Change workclass → Private (+-0.01, new prob: 0.09)
- Change education → 11th (+-0.02, new prob: 0.08)
- Change hours.per.week → 40 (+-0.02, new prob: 0.07)

🚀 Multi-Feature Optimization:

Best Combined Strategy: ('education', 'workclass') (+-0.02, new prob: 0.07)

📊 Structured Impact Summary:



,feature,recommended_value,new_probability,improvement
0,workclass,Private,0.085637,-0.008620
1,education,11th,0.078471,-0.015786
2,hours.per.week,40,0.071446,-0.022811



📊 Multi-Feature Combination Summary:



,change_set,new_probability,improvement
0,"(education, workclass)",0.071181,-0.023075
1,"(workclass, hours.per.week)",0.064763,-0.029494
2,"(education, hours.per.week)",0.059230,-0.035026


In [92]:
full_income_analysis(0, model, preprocessor, X, y)


📊 FULL INCOME ADVISORY REPORT

Predicted Probability of >50K: 0.44

🔍 Profile Alignment:

✔ education: aligned with high earners (HS-grad)
✖ occupation: differs from high earners (common: Exec-managerial)
✖ workclass: differs from high earners (common: Private)
✖ hours.per.week: differs from high earners (common: 38)

📈 Single-Feature Impact Analysis:

- Change occupation → Exec-managerial (+0.27, new prob: 0.72)
- Change workclass → Private (+0.08, new prob: 0.52)
- Change hours.per.week → 38 (+-0.01, new prob: 0.43)

🚀 Multi-Feature Optimization:

Best Combined Strategy: ('occupation', 'workclass') (+0.33, new prob: 0.78)

📊 Structured Impact Summary:



,feature,recommended_value,new_probability,improvement
0,occupation,Exec-managerial,0.716533,0.273092
1,workclass,Private,0.522250,0.078810
2,hours.per.week,38,0.428591,-0.014849



📊 Multi-Feature Combination Summary:



,change_set,new_probability,improvement
0,"(occupation, workclass)",0.776190,0.332749
1,"(occupation, hours.per.week)",0.704108,0.260667
2,"(workclass, hours.per.week)",0.507167,0.063726


In [93]:
full_income_analysis(1, model, preprocessor, X, y)


📊 FULL INCOME ADVISORY REPORT

Predicted Probability of >50K: 0.59

🎉 Your profile aligns with higher-income earners.
No immediate structural changes required.




In [95]:
full_income_analysis(2, model, preprocessor, X, y)


📊 FULL INCOME ADVISORY REPORT

Predicted Probability of >50K: 0.36

🔍 Profile Alignment:

✔ education: aligned with high earners (Some-college)
✖ occupation: differs from high earners (common: Adm-clerical)
✖ workclass: differs from high earners (common: Private)
✔ hours.per.week: aligned with high earners (40)

📈 Single-Feature Impact Analysis:

- Change occupation → Adm-clerical (+0.08, new prob: 0.44)
- Change workclass → Private (+0.08, new prob: 0.44)

🚀 Multi-Feature Optimization:

Best Combined Strategy: ('occupation', 'workclass') (+0.16, new prob: 0.52)

📊 Structured Impact Summary:



,feature,recommended_value,new_probability,improvement
0,occupation,Adm-clerical,0.444961,0.082546
1,workclass,Private,0.438161,0.075746



📊 Multi-Feature Combination Summary:



,change_set,new_probability,improvement
0,"(occupation, workclass)",0.523786,0.161371


## Step 5

In [69]:
young_worker = {
    "age": 25,
    "workclass": "Private",
    "education": "HS-grad",
    "education.num": 9,
    "marital.status": "Never-married",
    "occupation": "Handlers-cleaners",
    "relationship": "Not-in-family",
    "race": "White",
    "sex": "Male",
    "capital.gain": 0,
    "capital.loss": 0,
    "hours.per.week": 35,
    "native.country": "United-States"
}

In [ ]:
mid_career = { 
    "age": 48, 
    "workclass": "Private", 
    "education": "Bachelors", 
    "education.num": 13, 
    "marital.status": "Never-married", 
    "occupation": "Tech-support", 
    "relationship": "Unmarried", 
    "race": "White", 
    "sex": "Male", 
    "capital.gain": 0, 
    "capital.loss": 0, 
    "hours.per.week": 40, 
    "native.country": "United-States" 
    }

In [74]:
recommend_user_multi(young_worker, model, preprocessor, X, y)


📊 Income Recommendation Analysis (Multi-Feature)

Predicted Probability of >50K: 0.01

🔎 Ranked Recommendations (Best First):

1. occupation→Sales, hours.per.week→40
   ➜ New probability: 0.03 (+0.02)

2. occupation→Sales
   ➜ New probability: 0.03 (+0.02)

3. hours.per.week→40
   ➜ New probability: 0.01 (+0.00)



,change_set,features_changed,new_probability,improvement
0,"occupation→Sales, hours.per.week→40","(occupation, hours.per.week)",0.031978,0.021527
1,occupation→Sales,"(occupation,)",0.027621,0.017170
2,hours.per.week→40,"(hours.per.week,)",0.012134,0.001683


In [75]:
recommend_user_multi(mid_career, model, preprocessor, X, y)


📊 Income Recommendation Analysis (Multi-Feature)

Predicted Probability of >50K: 0.28

🔎 Ranked Recommendations (Best First):

1. occupation→Prof-specialty
   ➜ New probability: 0.40 (+0.12)



,change_set,features_changed,new_probability,improvement
0,occupation→Prof-specialty,"(occupation,)",0.400029,0.122663
